In [143]:
from abc import ABCMeta, abstractmethod
import six
import numpy as np
K_INIT_LIVING_DAYS = 27375  #每个人的平均寿命期望

In [144]:
class Person(object):
    """
    人类
    """
    def __init__(self):
        self.living = K_INIT_LIVING_DAYS    #初始化人平均能活的寿命
        self.happiness = 0                  #初始化幸福指数
        self.wealth = 0                     #初始化财富值
        self.fame = 0                       #初始化名望权力 
        self.living_day = 0                 #活着的第几天
    def live_one_day(self,seek):
        """
        每天只能seek一次，这个seek决定了今天追求的是什么，得到了什么
        seek的类型属于下面的BaseSeekDay
        :param seek:
        :return:
        """
        living_consumption, happiness, wealth, fame = seek.do_seek_day()#调用每个独特BaseSeekDay都会实现的do_seek_day得到今天收获
        self.living -= living_consumption     #每天减去生命消耗
        self.happiness += happiness       #通过seek得到的幸福指数积累
        self.wealth += wealth             #通过seek得到的财富指数积累
        self.fame += fame                 #通过seek得到的名望权力积累
        self.living_day += 1              #活完了一天

class BaseSeekDay(six.with_metaclass(ABCMeta, object)):
    def __init__(self):
        self.living_consume = 0      #每种追求每天消耗生命的常数
        self.happiness_base = 0      #每种追求每天幸福指数的常数
        self.wealth_base = 0         #每种追求每天财富积累的常数
        self.fame_base = 0           #每种追求每天名望权力积累的常数
        self.living_factor = [0]     #每种追求每天消耗生命的可变因素序列
        self.happiness_factor = [0]  #每种追求每天幸福指数的可变因素序列
        self.wealth_factor = [0]     #每种追求每天追求财富积累的可变因素序列
        self.fame_factor = [0]       #每种追求每天追求名望权力的可变因素序列
        self.do_seek_day_cnt = 0     #一生追求了多少天了
        self._init_self()            
    @abstractmethod
    def _init_self(self, *args, **kwargs):       #子类必须实现，设置生命消耗的常数，幸福指数常数等
        pass
    @abstractmethod
    def _gen_living_days(self, *args, **kwargs):  #子类必须实现，设置可变因素序列
        pass
    def do_seek_day(self):
        """
        每一天的追求具体seek
        """
        #生命消耗=消耗常数*可变序列
        if self.do_seek_day_cnt >= len(self.living_factor):
            consume_living = self.living_factor[-1] * self.consume_living #如果超出可变序列的长度，就按最后一个factor来算（递减）
        else:
            consume_living = self.living_factor[self.do_seek_day_cnt] * self.consume_living
        
        if self.do_seek_day_cnt >= len(self.happiness_factor):
            happiness = self.happiness_factor[-1] * self.happiness_base
        else:
            happiness = self.happiness_factor[self.do_seek_day_cnt] * self.happiness_base
        
        if self.do_seek_day_cnt >= len(self.wealth_factor):
            wealth = self.wealth_factor[-1] * self.wealth_base
        else:
            wealth = self.wealth_factor[self.do_seek_day_cnt] * self.wealth_base
        
        if self.do_seek_day_cnt >= len(self.fame_factor):
            fame = self.fame_factor[-1] * self.fame_base
        else:
            fame = self.fame_factor[self.do_seek_day_cnt] * self.fame_base
        
        self.do_seek_day_cnt += 1
        return consume_living, happiness, wealth, fame
    

In [145]:
def regular_mm(group):
    return (group - group.min()) / (group.max() - group.min())

In [146]:
class HealthSeekDay(BaseSeekDay):
    def _init_self(self):
        self.consume_living = 1
        self.happiness_base = 1
        self._gen_living_days()
    def _gen_living_days(self):
        days = np.arange(1,12000)
        living_days = np.sqrt(days)
        self.living_factor = regular_mm(living_days) * 2 - 1
        self.happiness_factor = regular_mm(days)[::-1]

In [147]:
me = Person()
seek_health = HealthSeekDay()
while me.living > 0:
    me.live_one_day(seek_health)
print('只追求健康长寿快乐活了{}年，幸福指数{},积累财富{},名望权力{}'.format(round(me.living_day / 365, 2), round(me.happiness, 2), me.wealth, me.fame))

只追求健康长寿快乐活了97.12年，幸福指数5999.5,积累财富0,名望权力0


In [148]:
class StockSeekDay(BaseSeekDay):
    def _init_self(self, show = False):
        self.consume_living = 2
        self.happiness_base = 0.5
        self.wealth_base = 10
        self._gen_living_days()
    def _gen_living_days(self):
        days = np.arange(1, 10000)
        living_days = np.sqrt(days)
        self.living_factor = regular_mm(living_days)
        happiness_days = np.power(days, 4)
        self.happiness_factor = regular_mm(happiness_days)[::-1]
        self.wealth_factor = self.living_factor

In [149]:
me = Person()
seek_stock = StockSeekDay()
while me.living > 0:
    me.live_one_day(seek_stock)
print('只追求财富活了{}年，幸福指数{},积累财富{},名望权力{}'.format(round(me.living_day / 365, 2), round(me.happiness, 2), round(me.wealth,2), me.fame))

只追求财富活了46.72年，幸福指数1000.15,积累财富136878.35,名望权力0


In [150]:
class FameSeekDay(BaseSeekDay):
    def _init_self(self):
        self.consume_living = 3
        self.happiness_base = 0.6
        self.fame_base = 10
        self._gen_living_days()
    def _gen_living_days(self):
        days = np.arange(1,12000)
        living_days = np.sqrt(days)
        self.living_factor = regular_mm(living_days)
        happiness_days = np.power(days,2)
        self.happiness_factor = regular_mm(happiness_days)[::-1]
        self.fame_factor = self.living_factor

In [151]:
me = Person()
seek_fame = FameSeekDay()
while me.living > 0:
    me.live_one_day(seek_fame)
print('只追求名望权力活了{}年，幸福指数{},积累财富{},名望权力{}'.format(round(me.living_day / 365, 2), round(me.happiness, 2), round(me.wealth,2), round(me.fame,2)))

只追求名望权力活了36.06年，幸福指数2400.1,积累财富0,名望权力91259.86


In [152]:
def my_life(weights):
    seek_health = HealthSeekDay()
    seek_stock = StockSeekDay()
    seek_fame = FameSeekDay()
    seek_list = [seek_health, seek_stock, seek_fame]
    me = Person()
    seek_choice = np.random.choice([0, 1, 2], 80000, p=weights)
    while me.living > 0:
        seek_ind = seek_choice[me.living_day]
        seek = seek_list[seek_ind]
        me.live_one_day(seek)
    return round(me.living_day / 365, 2), round(me.happiness, 2), round(me.wealth, 2), round(me.fame, 2)

In [153]:
living_day, happiness, wealth, fame = my_life([0.4, 0.3, 0.3])
print('活了{}年，幸福指数{}，积累财富{}，名望权力{}'.format(living_day, happiness, wealth, fame))

活了77.03年，幸福指数9314.39，积累财富51185.24，名望权力46191.54


In [154]:
result = []
for _ in range (2000):
    weights = np.random.random(3)
    weights /= np.sum(weights)
    result.append((weights, my_life(weights)))

In [156]:
sorted_scores = sorted(result, key=lambda x:x[1][1], reverse=True)
living_day, happiness, wealth, fame = my_life(sorted_scores[0][0])
print('活了{}年，幸福指数{}，积累财富{}，名望权力{}'.format(living_day, happiness, wealth, fame))

活了75.78年，幸福指数9371.97，积累财富29766.41，名望权力58202.7
